<div class="alert alert-block alert-success">
    <b><h2>Eksik Değerler (Missing Value)</h2></b> 
    <ul> Oluşturulan model ile bu değerlerin durumunu analiz ediyor olacağız. İlk başta sileceğiz sonra yerini dolduracağız ve modelin buna tepkisini ölçeceğiz.
    <br><li> Veri seti içerisindeki eksik değerleri ifade etmektedir.
        <li> Bir model eğitirken boş değerler içeren verilerden bir şekilde kurtulmamız gerekmektedir.
        <li> Bunun için farklı yakşalımlar sergileyebiliriz
    </ul>
        <img src="https://miro.medium.com/max/2816/1*MiJ_HpTbZECYjjF1qepNNQ.png"  width=400/>
</div>

#### Bu bölümde işlenecek konular

1. [RandomForestRegression modeli fonksiyon tanımlama](#bir)
2. [Boş değer içeren sütunların bulunması](#iki)
3. [Boş değer içeren sütunların düşürülmesi](#uc)
4. [Train test diye veri setini ayırma](#dort)
5. [Boş değerleri düşürüp eğitme ve sonuç](#bes)
6. [Boş değerleri SimpleImputer ile doldurma](#altı)
7. [Boş değerleri ortalama ile doldurduktan sonra eğitim ve sonuç](#yedi)

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

In [3]:
data=pd.read_csv('./Data/melbourne-housing-snapshot/melb_data.csv')

In [4]:
data.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


In [5]:
data.columns

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')

In [7]:
y = data.Price

In [85]:
# object olan satırlar exclude ediliyor ve sadece numeric değer içeren satırlar X içerisine aktarılıyor
X=data.select_dtypes(exclude=["object"])
X=X.drop(columns="Price")

In [86]:
X.head()

,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
0,2,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,NaN,-37.7996,144.9984,4019.0
1,2,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,-37.8079,144.9934,4019.0
2,3,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,-37.8093,144.9944,4019.0
3,3,2.5,3067.0,3.0,2.0,1.0,94.0,NaN,NaN,-37.7969,144.9969,4019.0
4,4,2.5,3067.0,3.0,1.0,2.0,120.0,142.0,2014.0,-37.8072,144.9941,4019.0


<div id="bir"  style="color:blue;"> 1. RandomForestRegressor modeli oluşturma</div>
<ul>
      <li>Model için n_estimators ile ağaç sayısı belirlenir</li>
    <li>random_state=0 her çalıştırmada verinin her sefer aynı şekilde ayrılması sağlanır <i> from sklearn.ensemble import RandomForestRegressor </i></li>
    <li>Mutlak hata oranını ölçüyoruz ve geri dönderiyoruz
</ul>

In [87]:
def score_dataset(X_train,X_test,y_train,y_test):
    model=RandomForestRegressor(n_estimators=10,random_state=0)
    model.fit(X_train,y_train)
    pred=model.predict(X_test)
    return mean_absolute_error(y_test,pred)


In [88]:
for item in X.columns:
    if X[item].isnull().any():
        print(item)

Car
BuildingArea
YearBuilt


<div id="iki"  style="color:blue;"> 2. Boş değer içeren sütunların bulunması</div>
<ul>
      <li>Yukarıda NaN içeren sütunlar bulundu</li>
    <li>NaN içeren sütunlar değişkene aktarılır</li>
</ul>

In [89]:
nan_value_column=[item for item in X.columns if X[item].isnull().any()]

<div id="uc"  style="color:blue;"> 3. Boş değerlerin veri setinden çıkarılması</div>
<ul>
      <li>NaN içeren değerler drop edilir</li>
    <li> axis=1 sütun, axis=0 satır demek. Burda sütunları düşürüyoruz</li>
</ul>

In [90]:
X_=X.drop(nan_value_column,axis=1)

In [91]:
X_.head()

,Rooms,Distance,Postcode,Bedroom2,Bathroom,Landsize,Lattitude,Longtitude,Propertycount
0,2,2.5,3067.0,2.0,1.0,202.0,-37.7996,144.9984,4019.0
1,2,2.5,3067.0,2.0,1.0,156.0,-37.8079,144.9934,4019.0
2,3,2.5,3067.0,3.0,2.0,134.0,-37.8093,144.9944,4019.0
3,3,2.5,3067.0,3.0,2.0,94.0,-37.7969,144.9969,4019.0
4,4,2.5,3067.0,3.0,1.0,120.0,-37.8072,144.9941,4019.0


<div id="dort"  style="color:blue;"> 4. Train test diye veri setini ayırma</div>
<ul>
    <li>Veriyi doğrulama yapmak için parçalışıyoruz. Verinin %80 train, %20 test için ayrılıyor
    <li> random_state=0 ile her seferinde verinin aynı parçalanması sağlanır
    <li>shuffle=True seçilmezse veriyi en baştan %80 ve geri kalanını %20 test diye ayırır. Shuffle ile veri setinin karıştırılığ parçalanmasını sağlar
</ul>

In [110]:
def train_test(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=2,shuffle=True)
    return X_train, X_test, y_train, y_test

In [111]:
X_train, X_test, y_train, y_test=train_test(X_,y)

In [112]:
X_train

,Rooms,Distance,Postcode,Bedroom2,Bathroom,Landsize,Lattitude,Longtitude,Propertycount
6941,2,12.4,3060.0,2.0,1.0,621.0,-37.71640,144.96660,5070.0
12969,4,7.5,3040.0,4.0,4.0,543.0,-37.74633,144.89644,9264.0
10653,5,27.0,3196.0,5.0,6.0,900.0,-38.04105,145.12160,2546.0
11652,3,16.1,3111.0,3.0,1.0,726.0,-37.80279,145.19974,4790.0
4871,3,4.5,3181.0,3.0,2.0,112.0,-37.85170,145.00560,7717.0
...,...,...,...,...,...,...,...,...,...
2514,2,5.8,3078.0,2.0,1.0,0.0,-37.78450,145.01050,2970.0
11798,2,2.7,3141.0,2.0,1.0,0.0,-37.83564,144.97729,14887.0
6637,3,7.0,3013.0,3.0,2.0,152.0,-37.82280,144.88330,6543.0
2575,3,3.5,3068.0,3.0,3.0,188.0,-37.78290,144.97880,6244.0


<div id="bes"  style="color:blue;"> 5. Boş değerlerin düşürüldüğü veri setini eğitme ve mutlak hata hesaplama</div>
<ul>
      <li>NaN değerleri içeren sütunları düşürdükten sonra eğitim </li>
</ul>

In [94]:
print("Boş değer içeren sütunlar silinmiştir")
print(score_dataset(X_train,X_test,y_train,y_test))

Boş değer içeren sütunlar silinmiştir
190103.4833456063


<div id="altı"  style="color:blue;"> 6. Boş değerleri SimpleImputer ile doldurma</div>
<ul>
    <li><b>SimpleImputer: </b>Boş değerleri ayarlanan parametre değerleri ile kolayca değiştirmeye yaramaktadır.</li>
    <li> Bazı önemli parametreleri</li>
    <ul>
        <li><b>missing_values: </b> Hangi verinin değiştirileceği girilir. Default olarak np.nan (NaN) değeridir
        <li><b>strategy: </b> Boş değerlere nasıl müdahele edileceği
            <ul>
                <li><b>mean :</b> Boş bulunan sütundaki ortalamayı alır boş değerleri doldurur
                <li><b>median :</b> Sütundaki ortanca değer ile değiştirir
                <li><b>most_frequent :</b> En sık kullanılan değer ile değiştirir
            </ul>
    </ul>
</ul>

In [96]:
from sklearn.impute import SimpleImputer

In [97]:
X.isnull().sum()

Rooms               0
Distance            0
Postcode            0
Bedroom2            0
Bathroom            0
Car                62
Landsize            0
BuildingArea     6450
YearBuilt        5375
Lattitude           0
Longtitude          0
Propertycount       0
dtype: int64

<ul>
    <li><b>mean :</b> SimpleImputer strategy="mean" ile sütundaki ortalamaya göre doldurma işlemi gerçekleştireceğiz
</ul>

In [99]:
imp=SimpleImputer(strategy="mean")

- pd.Dataframe ile dönüştürülen verinin dataframe formatında olmasını sağlıyoruz
- Yeni oluşturulan dataframe aynı sütunları atıyoruz

In [107]:
X_mean=pd.DataFrame(imp.fit_transform(X))
X_mean.columns=X.columns

In [108]:
X_mean.head()

,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
0,2.0,2.5,3067.0,2.0,1.0,1.0,202.0,151.96765,1964.684217,-37.7996,144.9984,4019.0
1,2.0,2.5,3067.0,2.0,1.0,0.0,156.0,79.00000,1900.000000,-37.8079,144.9934,4019.0
2,3.0,2.5,3067.0,3.0,2.0,0.0,134.0,150.00000,1900.000000,-37.8093,144.9944,4019.0
3,3.0,2.5,3067.0,3.0,2.0,1.0,94.0,151.96765,1964.684217,-37.7969,144.9969,4019.0
4,4.0,2.5,3067.0,3.0,1.0,2.0,120.0,142.00000,2014.000000,-37.8072,144.9941,4019.0


In [113]:
X_train,X_test,y_train,y_test=train_test(X_mean,y)

<div id="yedi"  style="color:blue;"> 7. Boş değerlerin ortalama ile doldurduktan sonra veri setini eğitme ve mutlak hata hesaplama</div>

In [114]:
print("Boş değerler SimpleImputer ile ortalama olarak dolduruldu")
print(score_dataset(X_train,X_test,y_train,y_test))

Boş değerler SimpleImputer ile ortalama olarak dolduruldu
183428.3329756645


## Boş değerlerin drop edildiği eğitime göre daha başarılı bir sonuç verdiği gözükmektedir
### <pre>Drop edilen             -> 190103 <li>Ortalama ile doldurunca -> 183428